## Goal:

The goal of this notebook is to extract data from the raw .csv files from each workflow, and combine them into two final dataframes, which we then parse to .json files again to save time later on (it's much faster to import a .json file than to re-generate the dataframes every time)

The two dataframes for which we'll create .json files:
- `df`: Dataframe where each row contains a single classification. 
- `df_votes`: Derived dataframe where each row corresponds to a single object, its (photometric) parameters derived by Venhola et al. using GALFIT, and the vote distributions for that object for every task.

## If you run this notebook:
The following dataframes will be created as .json files:
- `df_stacked.json`
    - Each row contains an individual classification. Contains all non-duplicate classifications from all workflows.
- `df_votes.json`
    - Each row contains a single object, its properties, and its vote distributions (counts per task, and percentage per answer per task)
- `df_stacked_exclude_n.json`, for $n \in \{5, 10, 25, 50\}$
    - Same as df_votes.json, except here we exclude the first $n$ votes per user

In [1]:
import sys
# need to append the directory where sf_lib is stored to system path 
#  this is one way to enable us to import sf_lib, where we store functions we use in multiple notebooks
sys.path.append('../..')

import numpy as np
import pandas as pd
import json

from sf_lib.df import (
    make_df_classify, 
)

from importlib import reload
import sf_lib.helpers
reload(sf_lib.helpers)
from sf_lib.helpers import (
    df_to_json,
    fleiss_kappa
)

import warnings
warnings.filterwarnings('ignore')

### Load auxiliary data:

In [2]:
# import dataframe with (GALFIT) object parameters (extracted from the likely ground truth catalogue dataset)
#  this .csv was created in one of the notebooks in the /analysis/catalogue directory of this project
object_info = pd.read_csv('../../catalogue/sf_spacefluff_object_data.csv', comment="#")

In [3]:
# uncomment to inspect the first row of the object_info dataframe for reference purposes
object_info.head(1)

,name,RA,DEC,Reff,r_mag,g_mag,axis_ratio,pos_angle,n,u,...,ge,re,ie,Reffe,r_mage,ne,C,mue_r,bae,RFF
0,UDGcand_0,56.232609,-35.335724,2.8934,19.8319,-1.0,0.6034,-27.775499,1.0271,22.426844,...,0.054816,0.046306,0.044244,0.332842,0.131324,0.117142,2.76421,23.585331,0.027318,-99.0


### Define tasks so we can load the dataframes properly

The three workflows differ in which questions ('tasks') they present to users.

<span style="color: red;"><strong>IMPORTANT!</strong></span> When we go to combine the three dataframes, we need to swap the names of the T1 and T2 columns in the `hardcore` workflow. For some reason, The question assigned to T1 in `Classify!` is assigned to T2 in `Hardcore`. If we don't swap them, we'll end up with useless data.

In [4]:
tasks_hardcore = [0, 2, 1, 3, 4, 5, 9]
tasks_classify = [0, 1]
tasks_onthego = [0]

# hardcore contains all tasks, so use this to generate the final dataframe.
#  classifications that are from other workflows will simply have NaN or None 
#  as values for tasks that aren't present in that workflow
task_strings = ['T{}'.format(t) for t in tasks_hardcore]

### Load dataframes

In [5]:
df_classify = make_df_classify('classify', tasks_classify)
df_hardcore = make_df_classify('hardcore', tasks_hardcore)
df_onthego = make_df_classify('onthego', tasks_onthego)

#### Swap T1 and T2 columns in `hardcore` workflow

In [6]:
df_hardcore[['T1', 'T2']] = df_hardcore[['T2', 'T1']]

### Stack the three dataframes

Note the following inconsistency: `onthego` formulates one of the _task 0_ answers as "Group of objects (cluster)", while `classify` and `hardcore` have it formulated with uppercase _C_: "Group of objects (Cluster)". 

__Fix this by just coercing all answers to lowercase:__

In [7]:
df = df_classify.append(df_hardcore).append(df_onthego)
df['T0'] = df['T0'].apply(lambda x: x.lower())

__Sort classifications by date__. This makes it easier to filter classifications made by the same user, of the same object, in multiple workflows (we will only keep the first classification of an object in cases where a user saw it in multiple workflows)

In [8]:
df = df.sort_values('created_at')

In [9]:
print('Total number of classifications across all three workflows:', df.shape[0])

Total number of classifications across all three workflows: 233375


## Find duplicate classifications of an object made by the same user across workflows

A 'duplicate' classification is one where a single user saw a single object (e.g. `UDGcand_001`) in multiple workflows (e.g. `Hardcore` and `Classify!`). The duplicates are the classifications after the one in the first workflow where the user saw that object.

For code clarity purposes, I won't be coding the most efficient way time-complexity wise. 

In [10]:
# slice dataframe columns we need for much faster indexing
groupby_user = df[['user_name', 'Filename', 'T0']].groupby('user_name')
unique_users = df['user_name'].unique()

In [11]:
# loop through groups to find users who made more classifications than they saw unique objects,
#  i.e. they saw at least one object multiple times
users_with_duplicates = []

for user in unique_users:
    classifications_by_user = groupby_user.get_group(user)
    objects_seen_by_user = classifications_by_user['Filename']
    
    if not objects_seen_by_user.shape[0] == objects_seen_by_user.unique().shape[0]:
        users_with_duplicates.append(user)

In [12]:
print('Total number of users with \'duplicate\' classifications:', len(users_with_duplicates))

Total number of users with 'duplicate' classifications: 233


### Generate a list with each user and the objects they saw multiple times:

In [13]:
duplicate_classifications = {user: [] for user in users_with_duplicates}

for user in users_with_duplicates:
    classifications_by_user = groupby_user.get_group(user)
    objects = classifications_by_user['Filename']
    
    objects_seen_by_user = []
    for obj in objects:
        if not obj in objects_seen_by_user:
            objects_seen_by_user.append(obj)
        else:
            duplicate_classifications[user].append(obj)      

### Loop through all duplicate classifications and extract classification_id of every classification where the user had already seen that object:

In [14]:
to_filter = []  # this will become a list of `classification_id`

for user, dupes in duplicate_classifications.items():
    objects_seen = []

    # query df by username and filename to get classification_ids of 'duplicates'
    vals = df.query("user_name == @user & Filename.isin(@dupes)")[['Filename', 'classification_id']].values
    for entry in vals:
        name, clas_id = entry
        if name in objects_seen:
            to_filter.append(clas_id)
        else:
            objects_seen.append(name)

## Filter all classifications where that user had already seen that object (in another workflow):

In [15]:
df = df.query("~classification_id.isin(@to_filter)")

In [16]:
print('Total number of classifications across all three workflows after filtering duplicates:', df.shape[0])

Total number of classifications across all three workflows after filtering duplicates: 223059


## Create `df_votes` dataframe as mentioned above:

In [17]:
def df_votes_with_vote_count(df_votes):
    'Append column containing the total number of votes an object received from users'
    df_votes.insert(1, 'vote_count', df_votes['T0'].apply(lambda x: sum(list(x.values()))))
    return df_votes

def df_votes_with_object_info(df_votes):
    'Assign (photometric) properties to each object by merging `df_votes` dataframe with `object_info` dataframe'
    df_votes = df_votes.merge(object_info, how='outer', on='name')
    df_votes = df_votes.query("~vote_count.isnull()")
    return df_votes

def get_answer_vote_percentage(row, unique_answer, decimal_places=1):
    '''
        Assign the percentage of votes given for the specified unique answer
        @param row: the task column (e.g. `T0`) of a single row from a `df_votes`-type dataframe
            where a task column looks like (using T0 as example) 
                {
                    'galaxy': 10,
                    'group of objects (cluster)': 2,
                    'something else/empty center': 0
                }
        @param unique_answer: one of the unique answers provided for a task (e.g. for task 0, one of the unique answers is `galaxy`)
            in the above T0 example, unique_answer may be either 'galaxy', 'group of objects (cluster)' or 'something else/empty center',
            but it could also be None
        @returns percentage of votes for the given answer, rounded to 1 decimal place by default
    '''
    
    none_count = row.get('None', 0)
    total_votes = sum(row.values())
    actual_votes = total_votes - none_count
    
    if actual_votes > 0:
        return round(100*row.get(unique_answer, 0)/actual_votes, decimal_places)

def df_votes_with_vote_percentages(df_votes, df):
    '''
        Assign vote percentages for every unique answer from each unique task to every object using get_answer_vote_percentage
        @param df_votes: `df_votes`-like dataframe (where every row contains one object, all its votes for each task, and its (photometric) properties)
        @param df: `df`-like dataframe (where every row contains one classification)
        @returns `df_votes` with new a new column containing the percentage of votes it received for every unique answer for each task
    '''
    for task in task_strings:
        
        for unique_answer in df.query("~{}.isnull()".format(task))[task].unique().tolist():
            if not task == 'T0':
                unique_answer = unique_answer.capitalize()
            df_votes["{} % {}".format(task, unique_answer.lower())] = df_votes[task].apply(lambda x: get_answer_vote_percentage(x,  unique_answer))
            
    return df_votes

def make_df_votes(df, task_strings):
    '''
        Convert a `df`-like dataframe into a `df_votes` dataframe, 
            where each row contains a single object and all its votes and (photometric) properties
        @param df: `df`-like dataframe (where each row contains one classification) 
        @param task_strings: list of all tasks for which to extract vote counts and percentages, e.g. ['T0', 'T1']
    '''
    groupby_name = df[['Filename', *task_strings]].groupby('Filename')
    
    vals_list = []
    for object_name in df['Filename'].unique().tolist():
        vals = { "name": object_name }
        for task in tasks_hardcore:
            t = 'T{}'.format(task)
            vals[t] = groupby_name.get_group(object_name)[t].value_counts().to_dict()

        vals_list.append(vals)

    df_votes = pd.DataFrame(vals_list)
    df_votes = df_votes_with_vote_count(df_votes)
    df_votes = df_votes_with_object_info(df_votes)
    df_votes = df_votes_with_vote_percentages(df_votes, df)
    
    return df_votes

##### Export `df_votes` to .json (each row contains all votes and properties of a single object) to json:

In [18]:
# hardcore contains all tasks, so use this to generate the final dataframe.
#  classifications that are from other workflows will simply have NaN or None 
#  as values for tasks that aren't present in that workflow
task_strings_hc = ['T{}'.format(t) for t in tasks_hardcore]

df_votes = make_df_votes(df, task_strings_hc)
df_to_json(df_votes, 'df_votes')

##### Export `df` to .json (each row contains a single classification. Contains all non-duplicate classifications from all workflows) to json:

In [19]:
df.reset_index(drop=True, inplace=True)
df_to_json(df, 'df_stacked')

---

## Some statistics from the filtering process above, like # of users that saw any object multiple times, number of classifications to be filtered out, etc.

In [20]:
print('Number of users that saw at least one object multiple times:', len(users_with_duplicates))  # Discovery: 233 users saw the same object multiple times across workflows.

duplicate_count = [len(objects) for [user, objects] in duplicate_classifications.items()] 

## uncomment to print the frequency of duplicate votes (first entry is # of duplicates seen by user, second is the amount of users that saw that many duplicate objects)
# duplicate_count_frequency = np.unique(duplicate_count, return_counts=True)
# print('[# of objects seen multiple times per user,  frequency]', '\n', np.array(duplicate_count_frequency).T)  # Discovery: there is one user that saw 4363 duplicate objects, and one that saw 2046 duplicates. What happened here?

Number of users that saw at least one object multiple times: 233


### Extract usernames of users that saw more than 1000 duplicates:

In [21]:
has_many_duplicates = list(filter(lambda entry: len(entry[1]) > 1000, duplicate_classifications.items()))
has_many_duplicates = [user[0] for user in has_many_duplicates]

#### Print number of votes cast per option for T0 (task 0) by these users:

In [22]:
for user in has_many_duplicates:
    print(df.query("user_name == @user")['T0'].value_counts().to_dict())
    print('# votes by user:', df.query("user_name == @user").shape[0])
    print('\n')

{'galaxy': 3422, 'group of objects (cluster)': 1724, 'something else/empty center': 805}
# votes by user: 5951


{'galaxy': 3837, 'group of objects (cluster)': 1233, 'something else/empty center': 690}
# votes by user: 5760




##### Print number of duplicate classifications that were filtered out:

In [23]:
print('Number of classifications filtered as duplicates:', len(to_filter))

Number of classifications filtered as duplicates: 10316


Discovery: Of the total 233375 classifications, there are 10316 classifications (approx. 4.4% of the total) made by users that had already seen that object at least once.

## Exclude first $n$ classifications per user:

In [24]:
def exclude_first_n(df, n):
    '''
        Filter df by excluding first `n` classifications done by each user
        @param df: input dataframe
        @param n (int): number of classifications per user to exclude
        @returns filtered dataframe, total number of excluded classifications
    '''
    groupby_user = df[['user_name', 'classification_id', 'created_at']].groupby('user_name')

    exclude_ids = []
    for name, group in groupby_user:
        ids_to_exclude = group['classification_id'].tolist()[:n]
        exclude_ids.append(ids_to_exclude)
    
    exclude_ids = np.concatenate(exclude_ids)
    return df.query('~classification_id.isin(@exclude_ids)'), exclude_ids.shape

In [25]:
def make_df_votes_exclude_n(df, task_strings, n):
    'Create and save to .json a version of df_votes with the first n classifications per user filtered out'
    
    df_exclude_n, num_excluded = exclude_first_n(df, n)
    
    df_votes_exclude_n = make_df_votes(df_exclude_n, task_strings)
    
    df_to_json(df_votes_exclude_n, 'df_votes_exclude_{}'.format(n))
    print('saved df_votes_exclude_{}.json'.format(n))
    print('Total number of classifications excluded: {}'.format(num_excluded))
    print('\n')
    return df_votes_exclude_n, num_excluded

In [26]:
exclude_counts = [5, 10, 25, 50, 250]

- Uncomment the next cell to create `df_votes` dataframes resulting from excluding the first $n$ classifications per user

In [27]:
# # create df_votes_exclude_`n`.json for various `n`
# #  running this cell takes a while, and we could speed it up by using dynamic programming, 
# #  but this is the most readable format and we should only need to run the cell once anyway

# for n in exclude_counts:
#     make_df_votes_exclude_n(df, task_strings_hc, n)

### Include _all_ votes, but only of power users (e.g. users that made at least 250 votes)

In [28]:
power_users = []
for name, group in df.groupby('user_name'):
    if group.shape[0] >= 250:
        power_users.append(name)

In [29]:
df_power_users = df.query('user_name.isin(@power_users)')

In [30]:
df_votes_power_users = make_df_votes(df_power_users, task_strings_hc)

In [31]:
df_to_json(df_votes_power_users, 'df_votes_power_users')

### Compute Fleiss $\kappa$ for task 0 for the dataframes excluding the first $n$ classifications per user for various $n$:

In [32]:
exclude_counts = [5, 10, 25, 50, 250]
dfs_votes_excluding_n = [pd.read_json('df_votes_exclude_{}.json'.format(n)) for n in exclude_counts]
dfs_excluding_n = [exclude_first_n(df, n)[0] for n in exclude_counts]

In [43]:
# print Fleiss kappa for task 0 while excluding various numbers of leading classifications per user

print('Exclude n | Fleiss kappa')
print('-'*30)
for i, (d, d_v) in enumerate(zip(dfs_excluding_n, dfs_votes_excluding_n)):
    print('{:9.0f} | {}'.format(exclude_counts[i], fleiss_kappa(d, d_v, 'T0')))

Exclude n | Fleiss kappa
------------------------------
        5 | 0.6736646178382614
       10 | 0.6783956776300358
       25 | 0.688200672137247
       50 | 0.7008247323197264
      250 | 0.7765682246740122
